# Calculating SHAP values

**ATTENTION:**

Notebook language: **Python**

In [ ]:
#!pip install numba --upgrade
#!pip install shap

## Loading model and data

In [ ]:
import pickle5 as pickle

In [ ]:
with open('./model/model.pickle', 'rb') as fp:
    model = pickle.load(fp) 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_preprocessed = pd.read_csv('./data/data_preprocessed.csv', index_col=0)
df_raw = pd.read_csv('./data/raw_data.csv', index_col=0)

In [ ]:
X_preprocessed = df_preprocessed.drop('status', axis=1)
X_raw = df_raw.drop('status', axis=1)

## Calculating predictions (background prediction)

In [ ]:
y_hat = pd.DataFrame(model.predict_proba(X_preprocessed)[:, 1])
y_hat.to_csv('./data/y_hat_full.csv')

## Creating explainer

In [ ]:
import shap
from skranger.utils.shap import shap_patch

In [ ]:
with shap_patch():
    explainer = shap.TreeExplainer(model)

## Functions to calculate explanations

In [ ]:
import os

path = './results'

if not os.path.exists(path):
    os.makedirs(path)
else:
    print("The folder already exists")

In [ ]:
from scripts.calculate_SHAP import extract_preprocessed__calculate__save

## Calculations

### Robert Lewandowski

#### Season 2021

In [ ]:
subset = X_raw[X_raw['player'] == 'Robert Lewandowski' & X_raw['season'] == 2021]

In [ ]:
extract_preprocessed__calculate__save(
    main_dir = './results', 
    task_hierarchy = ['lewandowski', 'season2021'],
    explainer = explainer, 
    subset = subset, 
    df_preprocessed = X_preprocessed
)